In [2]:
import os
import pandas as pd
import numpy as np
from astropy import units as u
from astropy.time import Time
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import stats
%matplotlib widget

#data = pd.read_csv('/Users/87steven/Documents/ASIAA/ACC variability/ACC Xmatch pushkarevkovalev_source_size.csv')
data = pd.read_csv('/Users/87steven/Documents/ASIAA/ACC variability new/BZCAT_Xmatch_ACC_csv.csv')

Name = data.name
cla = data.Class
RA = data.RA
DEC = data.DEC
data_flux = data.flux_ALMA
data_flux_err = data.errflux
data_band = data.band
data_freq = data.freq
obs_date = data.date
#data_b = data.gla_b
#data_l = data.gla_l
#data_2 = data.theta_2
#data_5 = data.theta_5
#obs_8 = data.theta_8
#data_15 = data.theta_15
#data_24 = data.theta_24
#obs_43 = data.theta_43

#######################################################################################################
### put name into a list
namelist = list(set(data.name))
name_index = [[np.nan]*1]*len(namelist) # len = 1051

### put index of each source into an array
for i in range(0, len(namelist)):
    #print(namelsit[i])
    name_index[i] =  np.where( (Name == namelist[i]).values == True )[0]
    

In [39]:
#######################################################################################################
### Save obseravtion date, and light curve data (band 3, 6, 7) into a CSV file 
### "ACC light curve flagged"
#######################################################################################################

def ClearData():
    data = {
    'mjd_3': [],
    'flux_3': [],
    'fluxerr_3': [],
    'mjd_6': [],
    'flux_6': [],
    'fluxerr_6': [],
    'mjd_7': [],
    'flux_7': [],
    'fluxerr_7': [],
      }
    return data

bandlist = list(set(data_band))
df = ClearData()

for i in range(0, len(namelist)): # len(namelist)
    
    df = ClearData()
    
    datalen = len(np.array(data_flux[name_index[i]]))
    print('i = ', i ,', Source name: ' +namelist[i], ', data length = ', datalen)
    
    Source_name = str(namelist[i])
    
    datalen_num = 50
    if datalen >= datalen_num:  
        
        index_each_band = [[np.nan]]*len(set(data_band)) # len(set(data_band)) = 7

        for k, band_num in enumerate( set(data_band) ):  
            #print(k, ',', band_num)
    
            # find "True" in specific band        
            aa = (data_band[name_index[i]] == band_num) # band = 3, 4, 6, 7, 8, 9, 10
            # find index of "True" value
            b = np.where(( aa == True))[0]
            # find true index of the band
            index_each_band[k] = np.array( aa.index[b] )
    
            if len(index_each_band[k]) == 0:
                index_each_band[k] = np.array([np.nan])
            
        for k in range(0, 7): # i = 0 => band 3, i = 1 => band 4, i = 2 => band 6, i = 3 => band 7, i = 4 => band 8, i = 5 => band 9, i = 6 => band 10
            if ~np.isnan(index_each_band[k][0]) == True:
                
                #print('k = ', k)
                
                # extract flux, flux error
                if k == 0: # band 3
                    # observation date format
                    band_date = obs_date[index_each_band[k]]
                    band_t = Time(list(band_date.values), format='isot', scale='utc')
                    date_3 = band_t.mjd
                    
                    flux_3 = np.array(data_flux[index_each_band[k]])
                    fluxerr_3 = np.array(data_flux_err[index_each_band[k]])    
                elif k == 2: # band 6
                    band_date = obs_date[index_each_band[k]]
                    band_t = Time(list(band_date.values), format='isot', scale='utc')
                    date_6 = band_t.mjd
                    
                    flux_6 = np.array(data_flux[index_each_band[k]])
                    fluxerr_6 = np.array(data_flux_err[index_each_band[k]])    
                elif k == 3: # band 7
                    band_date = obs_date[index_each_band[k]]
                    band_t = Time(list(band_date.values), format='isot', scale='utc')
                    date_7 = band_t.mjd
                    
                    flux_7 = np.array(data_flux[index_each_band[k]])
                    fluxerr_7 = np.array(data_flux_err[index_each_band[k]])
                
        print('flux_3 len=', len(flux_3), ', flux_6 len=', len(flux_6), ', flux_7 len=', len(flux_7))
           
        len_3 = len(flux_3)
        len_6 = len(flux_6)
        len_7 = len(flux_7)

        datalen = [len_3, len_6, len_7]

        max_len_ind = np.where(datalen == np.amax(datalen))[0][0]
        
        if len_3 < datalen_num:
            flux_3 = [np.nan]*50
            fluxerr_3 = [np.nan]*50
        if len_6 < datalen_num:
            flux_6 = [np.nan]*50
            fluxerr_6 = [np.nan]*50
        if len_7 < datalen_num:
            flux_7 = [np.nan]*50
            fluxerr_7 = [np.nan]*50
        
        if max_len_ind == 0: # band 3 have most data length
            for i in range(0, len_3):
                df['mjd_3'].append(date_3[i])
                df['flux_3'].append(flux_3[i])
                df['fluxerr_3'].append(fluxerr_3[i])
            for i in range(0, len_6):   
                df['mjd_6'].append(date_6[i])
                df['flux_6'].append(flux_6[i])
                df['fluxerr_6'].append(fluxerr_6[i])
            for i in range(0, len_7):
                df['mjd_7'].append(date_7[i])
                df['flux_7'].append(flux_7[i])
                df['fluxerr_7'].append(fluxerr_7[i])
                
            for i in range(len_6, len_3):   
                df['mjd_6'].append(np.nan)
                df['flux_6'].append(np.nan)
                df['fluxerr_6'].append(np.nan)
            for i in range(len_7, len_3):
                df['mjd_7'].append(np.nan)
                df['flux_7'].append(np.nan)
                df['fluxerr_7'].append(np.nan)
                
        elif max_len_ind == 1:  # band 6 have mpst data length
            for i in range(0, len_3):
                df['mjd_3'].append(date_3[i])
                df['flux_3'].append(flux_3[i])
                df['fluxerr_3'].append(fluxerr_3[i])
            for i in range(0, len_6):   
                df['mjd_6'].append(date_6[i])
                df['flux_6'].append(flux_6[i])
                df['fluxerr_6'].append(fluxerr_6[i])
            for i in range(0, len_7):
                df['mjd_7'].append(date_7[i])
                df['flux_7'].append(flux_7[i])
                df['fluxerr_7'].append(fluxerr_7[i])
                
            for i in range(len_3, len_6):   
                df['mjd_3'].append(np.nan)
                df['flux_3'].append(np.nan)
                df['fluxerr_3'].append(np.nan)
            for i in range(len_7, len_6):
                df['mjd_7'].append(np.nan)
                df['flux_7'].append(np.nan)
                df['fluxerr_7'].append(np.nan)
                
        elif max_len_ind == 2: # band 7 have mpst data length
            for i in range(0, len_3):
                df['mjd_3'].append(date_3[i])
                df['flux_3'].append(flux_3[i])
                df['fluxerr_3'].append(fluxerr_3[i])
            for i in range(0, len_6):   
                df['mjd_6'].append(date_6[i])
                df['flux_6'].append(flux_6[i])
                df['fluxerr_6'].append(fluxerr_6[i])
            for i in range(0, len_7):
                df['mjd_7'].append(date_7[i])
                df['flux_7'].append(flux_7[i])
                df['fluxerr_7'].append(fluxerr_7[i])
                
            for i in range(len_3, len_7):   
                df['mjd_3'].append(np.nan)
                df['flux_3'].append(np.nan)
                df['fluxerr_3'].append(np.nan)
            for i in range(len_6, len_7):
                df['mjd_6'].append(np.nan)
                df['flux_6'].append(np.nan)
                df['fluxerr_6'].append(np.nan)
        
    
        CSVfile = f'/Users/87steven/Documents/ASIAA/ACC variability new/ACC lightcurve flagged gt 50 points/'+Source_name+'.csv'   
        dff = pd.DataFrame(df)      
        dff.to_csv(CSVfile, index = False)
        
        print('================================================ File saved succfully ================================================')


i =  0 , Source name: J1147-6753 , data length =  36
i =  1 , Source name: J0113+0222 , data length =  14
i =  2 , Source name: J1220+3431 , data length =  4
i =  3 , Source name: J1959-4246 , data length =  4
i =  4 , Source name: J2011-1546 , data length =  11
i =  5 , Source name: J2202-2335 , data length =  2
i =  6 , Source name: J0613+1306 , data length =  4
i =  7 , Source name: J0948-3223 , data length =  2
i =  8 , Source name: J0509+1011 , data length =  11
i =  9 , Source name: J0126-2222 , data length =  5
i =  10 , Source name: J0902-1415 , data length =  1
i =  11 , Source name: J1228-0304 , data length =  1
i =  12 , Source name: J0739+0137 , data length =  19
i =  13 , Source name: J2327+1524 , data length =  4
i =  14 , Source name: J1603-4904 , data length =  26
i =  15 , Source name: J0856+2111 , data length =  5
i =  16 , Source name: J0216-0118 , data length =  1
i =  17 , Source name: J0903-1721 , data length =  1
i =  18 , Source name: J1002+1216 , data length = 

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]